In [304]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import cvxpy as cp
from datetime import datetime, timedelta

In [305]:
tickers = ['SBIN.NS','AAPL','GLD','QQQ','EEM']
start_date = '2015-08-13'
end_date = datetime.today().strftime('%Y-%m-%d')
df = yf.download(tickers,start_date,end_date, interval='1d')['Close'].dropna()

/var/folders/y7/hf6140q96774323j7d756khw0000gn/T/ipykernel_47328/2312300027.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  5 of 5 completed


In [306]:
log_returns = np.log(df/df.shift(1)).dropna()
mean_returns = log_returns.mean()*252
cov_matrix = log_returns.cov()*252
cor_matrix = log_returns.corr()
# print(log_returns,mean_returns,cov_matrix)
print(mean_returns)
print(cor_matrix)

Ticker
AAPL       0.231525
EEM        0.059413
GLD        0.111312
QQQ        0.182837
SBIN.NS    0.132983
dtype: float64
Ticker       AAPL       EEM       GLD       QQQ   SBIN.NS
Ticker                                                   
AAPL     1.000000  0.550304  0.028145  0.805790  0.113931
EEM      0.550304  1.000000  0.167814  0.717427  0.257006
GLD      0.028145  0.167814  1.000000  0.052470 -0.013817
QQQ      0.805790  0.717427  0.052470  1.000000  0.135277
SBIN.NS  0.113931  0.257006 -0.013817  0.135277  1.000000


In [307]:
np.random.seed()

n_portfolio =5000
n_asset =len(tickers)

results={
    'Returns':[],
    'Volatility':[],
    'Sharpe Ratio':[],
    'Weights':[]
}

for _ in range(n_portfolio):
    weights = np.random.random(n_asset).round(3)
    weights /= np.sum(weights).round(3)
    portfolio_return =np.dot(weights,mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix,weights)))
    sharpe_ratio = portfolio_return/portfolio_volatility
    if portfolio_volatility <0.25:
        results['Returns'].append(portfolio_return)
        results['Volatility'].append(portfolio_volatility)
        results['Sharpe Ratio'].append(sharpe_ratio)
        results['Weights'].append(weights)

portfolios_df =pd.DataFrame(results)
portfolios_df.head()

,Returns,Volatility,Sharpe Ratio,Weights
0,0.135299,0.137196,0.986176,"[0.14138678223185266, 0.16684723726977246, 0.3..."
1,0.144417,0.144214,1.001408,"[0.22725349132458741, 0.14007617435463396, 0.3..."
2,0.150349,0.166764,0.901566,"[0.22371514947934165, 0.11420893516963387, 0.2..."
3,0.121407,0.136897,0.886850,"[0.13217905405405406, 0.28040540540540543, 0.3..."
4,0.127502,0.189224,0.673813,"[0.22356143079315705, 0.38297045101088645, 0.0..."


In [308]:
fig = px.scatter(portfolios_df,x='Volatility',y='Returns',color='Sharpe Ratio',hover_data=['Weights'])
fig.update_layout(title='Raw Efficient Frontier',xaxis_title='Volatility',yaxis_title='Returns')
fig.show()

In [309]:
max_sharpe_idx = portfolios_df['Sharpe Ratio'].idxmax()

In [310]:
portfolios_df.iloc[max_sharpe_idx]['Weights']

array([0.1162492 , 0.01284522, 0.55619782, 0.24213231, 0.07257547])

In [311]:
fig = px.scatter(portfolios_df,x='Volatility',y='Returns',color='Sharpe Ratio',hover_data=['Weights'])
fig.add_scatter(
    x=[portfolios_df['Volatility']],  # Extract value as list
    y=[portfolios_df['Returns']],
    mode='markers',
    name='max Sharpe Portfolio',
    marker=dict(size=14, color='red',symbol='star')
)
fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.show()


In [312]:
fig_weights = px.bar(
    x=tickers,
    y=portfolios_df.iloc[max_sharpe_idx]['Weights'],
    opacity=0.7,
    title='Optimal Portfolio Weights'
)
fig_weights.show()

In [313]:
mu = mean_returns.to_numpy()
Sigma = cov_matrix.to_numpy()
n_asset = len(mu)

w=cp.Variable(n_asset)

portfolio_return = mu@ w
portfolio_volatility = cp.quad_form(w, Sigma)

constraints = [
    cp.sum(w) == 1,
    w>=0.1,
    portfolio_volatility <= 1
]

prob = cp.Problem(cp.Maximize(portfolio_return), constraints)
prob.solve()

opt_weights = w.value

print("Soler Optimal Weights:", opt_weights)
print("Optimal Portfolio Return:", portfolio_return.value)
print("Optimal Portfolio Volatility:", portfolio_volatility.value)

for i in range(n_asset):
    print(f"{tickers[i]}: {round(opt_weights[i],2)}")
# print(portfolio_return, portfolio_volatility)
# weights = portfolios_df.iloc[max_sharpe_idx]['Weights'].to_numpy()

Soler Optimal Weights: [0.6 0.1 0.1 0.1 0.1]
Optimal Portfolio Return: 0.18756965530596892
Optimal Portfolio Volatility: 0.04839243109375585
SBIN.NS: 0.6
AAPL: 0.1
GLD: 0.1
QQQ: 0.1
EEM: 0.1
